In [1]:
import pandas as pd
import numpy as np
from mapping import Mapping

fig_prefix = "../figuires/2018-07-22-jw-weaselball_analysis"
data_prefix = "../data/2018-07-22-jw-weaselball_analysis_"

In [2]:
FLOAT_ERROR_TOLERANCE = 0.00000000001 #See IEEE 754 for why a floating point is never perfect
df_strings = ['../data/08-11-2018_17-59-32.csv','../data/08-11-2018_17-59-39.csv' ]
frames = []
for csv in df_strings:
    temp = pd.read_csv(csv,index_col=False )
    frames.append(temp)
df = pd.concat(frames,ignore_index=True)
print(df.shape)
df = df.drop(columns=['Time'])
df = df.dropna() #Get rid of any rows with NA in it.
df = df.apply(pd.to_numeric)
df.head(10)

(19302242, 8)


,ID,X,Y,Yaw,ResetID,checkCorrectness,NumberOfWalls
0,0,0.000000,0.000000,0.00000,1,0,0
1,0,-0.466000,-0.279600,-2.29649,2,1,0
2,0,-0.466001,-0.279601,-2.29649,2,1,0
3,0,-0.466001,-0.279601,-2.29649,2,1,0
4,0,-0.466001,-0.279601,-2.29649,2,1,0
5,0,-0.466001,-0.279601,-2.29649,2,1,0
6,0,-0.466001,-0.279601,-2.29649,2,1,0
7,0,-0.466001,-0.279601,-2.29649,2,1,0
8,0,-0.466001,-0.279601,-2.29649,2,1,0
9,0,-0.466001,-0.279601,-2.29649,2,1,0


In [3]:
df.tail(10)

,ID,X,Y,Yaw,ResetID,checkCorrectness,NumberOfWalls
19302232,0,0.311417,0.486632,1.19492,18,1,0
19302233,0,0.311421,0.486478,1.19486,18,1,0
19302234,0,0.311424,0.486322,1.19479,18,1,0
19302235,0,0.311427,0.486168,1.19472,18,1,0
19302236,0,0.311431,0.486016,1.19465,18,1,0
19302237,0,0.311432,0.485866,1.19458,18,1,0
19302238,0,0.311431,0.485714,1.19451,18,1,0
19302239,0,0.311430,0.485563,1.19445,18,1,0
19302240,0,0.311434,0.485405,1.19438,18,1,0
19302241,0,0.311436,0.485250,1.19432,18,1,0


In [4]:
#Sample the data
SAMPLING_RATE = 25 #Keep 1 row for every SAMPLING_RATE
df_sampled = df.iloc[::SAMPLING_RATE,:]
print("Size of new DF is {}".format(df_sampled.shape))
df_sampled.head(10)

Size of new DF is (772090, 7)


,ID,X,Y,Yaw,ResetID,checkCorrectness,NumberOfWalls
0,0,0.000000,0.000000,0.00000,1,0,0
25,0,-0.466041,-0.279525,-2.29645,2,1,0
50,0,-0.466791,-0.277501,-2.29580,2,1,0
75,0,-0.466928,-0.275418,-2.29486,2,1,0
100,0,-0.466971,-0.273585,-2.29393,2,1,0
125,0,-0.466862,-0.272273,-2.29416,2,1,0
150,0,-0.465894,-0.271019,-2.29774,2,1,0
175,0,-0.462900,-0.270482,-2.30444,2,1,0
200,0,-0.458288,-0.270147,-2.31126,2,1,0
225,0,-0.452599,-0.270172,-2.31608,2,1,0


In [5]:
#Break data into 3 parts. S = {Near 2 Walls, Near 1 Wall, Near No Walls}
#df2 = df_sampled.loc[df_sampled['ResetID'] % 3 == 2]
#df1 = df_sampled.loc[df_sampled['ResetID'] % 3 == 1]
#df0 = df_sampled.loc[df_sampled['ResetID'] % 3 == 0]
#df = df0

In [6]:
#Clean up the data
df_clean = df_sampled.copy()

df_clean.index = range(df_clean.shape[0])
#When the gazebo run it may collect some data of the robots when they aren't moving for the first few 50 or so samples.
#TODO
df_clean.head()

,ID,X,Y,Yaw,ResetID,checkCorrectness,NumberOfWalls
0,0,0.000000,0.000000,0.00000,1,0,0
1,0,-0.466041,-0.279525,-2.29645,2,1,0
2,0,-0.466791,-0.277501,-2.29580,2,1,0
3,0,-0.466928,-0.275418,-2.29486,2,1,0
4,0,-0.466971,-0.273585,-2.29393,2,1,0


In [7]:
#Clean up the data

#Shift Yaw to go from 0 to 2pi, so just add PI since it currently goes for -pi to pi

df_clean['Yaw'] += np.pi
if(df_clean['Yaw'].max() > 2 * np.pi or df_clean['Yaw'].min() < 0):
    print("[ERROR] Cleaning Yaw failed. Make sure range is from 0 and 2 * pi")
    print("Yaw = ({} - {})".format(df_clean['Yaw'].min(), df_clean['Yaw'].max()))


df_clean.head()

,ID,X,Y,Yaw,ResetID,checkCorrectness,NumberOfWalls
0,0,0.000000,0.000000,3.141593,1,0,0
1,0,-0.466041,-0.279525,0.845143,2,1,0
2,0,-0.466791,-0.277501,0.845793,2,1,0
3,0,-0.466928,-0.275418,0.846733,2,1,0
4,0,-0.466971,-0.273585,0.847663,2,1,0


In [8]:
#Clean up the data

#Shift X and Y over so that way it could be made easier to use. Currently the world reference is at the center of the "play area"
LENGTH_OF_BOX = 1.127 #This can be obtained from the .sdf file of the weazelball enclosure in gazebo


df_clean['X'] += LENGTH_OF_BOX / 2
df_clean['Y'] += LENGTH_OF_BOX / 2
if (df_clean['X'].max() > LENGTH_OF_BOX or df_clean['Y'].max() > LENGTH_OF_BOX or df_clean['X'].min() < 0 or df_clean['Y'].min() < 0):
    print("[ERROR] Cleaning X/Y failed, Make sure the points are between 0 and LENGTH_OF_BOX")
    print("Y = ({} - {}) X = ({} to {})".format(df_clean['Y'].min(), df_clean['Y'].max(), df_clean['X'].min(), df_clean['X'].max()))
df_clean.head()


,ID,X,Y,Yaw,ResetID,checkCorrectness,NumberOfWalls
0,0,0.563500,0.563500,3.141593,1,0,0
1,0,0.097459,0.283975,0.845143,2,1,0
2,0,0.096709,0.285999,0.845793,2,1,0
3,0,0.096572,0.288082,0.846733,2,1,0
4,0,0.096529,0.289915,0.847663,2,1,0


In [9]:
#Clean up the data

#Clean the time data since gazebo prints it weird...
#TODO

In [10]:
#Discretize the data
NUMBER_OF_SQUARES = 400 #This should be a square number to create equal sized squares.
RESOLUTION_OF_S1 = 0.7854 #This is used to discretize the yaw angle over 0 - 2*pi

df_discretized = df_clean.copy()
    
mappingBoxConstant = (NUMBER_OF_SQUARES ** (1/2.0)) / (LENGTH_OF_BOX)
for index, row in df_clean.iterrows():
    df_discretized.at[index, 'X'] = int(row['X'] * mappingBoxConstant)
    df_discretized.at[index, 'Y'] = int(row['Y'] * mappingBoxConstant)
    df_discretized.at[index, 'Yaw'] = int(row['Yaw'] / RESOLUTION_OF_S1) * RESOLUTION_OF_S1
        
df_discretized.describe()

,ID,X,Y,Yaw,ResetID,checkCorrectness,NumberOfWalls
count,772090.0,772090.000000,772090.000000,772090.000000,772090.000000,772090.000000,772090.000000
mean,0.0,9.465965,9.512027,2.837665,9.530934,0.999999,0.031059
std,0.0,5.793065,5.837756,1.785966,4.629763,0.001138,0.174288
min,0.0,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000
25%,0.0,4.000000,4.000000,1.570800,6.000000,1.000000,0.000000
50%,0.0,9.000000,9.000000,3.141600,10.000000,1.000000,0.000000
75%,0.0,15.000000,15.000000,4.712400,14.000000,1.000000,0.000000
max,0.0,18.000000,18.000000,5.497800,18.000000,1.000000,2.000000


In [11]:
#Verify Discretizing suceeded by checking that number of states generated is the number of states we expeted or less (Sometimes these things dont visit all states)

if (df_discretized['X'].max() > (NUMBER_OF_SQUARES ** (1/2.0)) or df_discretized['Y'].max() > (NUMBER_OF_SQUARES ** (1/2.0)) or df_discretized['X'].min() < 0 or df_discretized['Y'].min() < 0):
    print("[ERROR] Discretizing X/Y failed, Make sure the points are between 0 and (NUMBER_OF_SQUARES ** (1/2.0)")
    print("Y = ({} - {}) X = ({} to {})".format(df_clean['Y'].min(), df_clean['Y'].max(), df_clean['X'].min(), df_clean['X'].max()))


In [12]:
TRANSLATION_MATRIX_INITIAL_VALUE = 1
#Create the matrix representing the Markov Chain
#I am assuming we are discretizing the space into equal sized boxes
#The transition matrix A is of size 
#( # of states of X * # of states of Y * # of states of Yaw)
#( # of states of X and Y = mappingBoxConstant * LENGTH_OF_BOX )
#( # of states of Yaw = int(2*pi / RESOLUTION_OF_S1)+1)
number_of_x_states = mappingBoxConstant * LENGTH_OF_BOX
number_of_y_states = mappingBoxConstant * LENGTH_OF_BOX
number_of_s1_states = int(2*np.pi / RESOLUTION_OF_S1) + 1

n = int(number_of_x_states * number_of_y_states * number_of_s1_states)
print("[DEBUG] Size of n is {}".format(n))
translation_matrix = pd.DataFrame(0, index=range(n), columns=range(n))#We use 1 here to set the whole matrix elements to 1
translation_matrix.head()

[DEBUG] Size of n is 3200


,0,1,2,3,4,5,6,7,8,9,...,3190,3191,3192,3193,3194,3195,3196,3197,3198,3199
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:


#The formula for mapping a 3D array to 1D is
#(z * xMax * yMax) + (y * xMax) + x;
#https://stackoverflow.com/questions/7367770/how-to-flatten-or-index-3d-array-in-1d-array
X_MAX = NUMBER_OF_SQUARES ** (1/2.0)
Y_MAX = NUMBER_OF_SQUARES ** (1/2.0)
YAW_MAX = (2 * np.pi) / RESOLUTION_OF_S1
mapping = Mapping(X_MAX, Y_MAX, YAW_MAX)#Fill in the logical areas that the system can reach (For now I am assuming it can go up/down 2 yaw states or the surronding (x,y) blocks)

In [14]:
#HUERISTIC: Add a +1 to any logical possible state the structure would likely end up in.
#This lowers the amount of artifiical data in the matrix (most of which isnt needed)



for index, row in translation_matrix.iterrows():
    if(TRANSLATION_MATRIX_INITIAL_VALUE == 0):
        break
    (x,y,yaw) = mapping.map1Dto3D(index)
    #Generate all possible (x,y,yaw) permutations
    #I assume we can only move with 1 around (x,y) including diagnols and +- 2 yaw states
    VARIATION_OF_X = 1
    VARIATION_OF_Y = 1
    VARIATION_OF_YAW = 1
    possible_spots = []
    for x_p in range(-VARIATION_OF_X,VARIATION_OF_X+1):
        for y_p in range(-VARIATION_OF_Y,VARIATION_OF_Y+1):
            for yaw_p in range(-VARIATION_OF_YAW,VARIATION_OF_YAW+1):
                if(mapping.checkValid3DMap(x+x_p, y+y_p, yaw+yaw_p)):
                    possible_spots.append( (x+x_p, y+y_p, yaw+yaw_p) )
    for pose in possible_spots:
        translation_matrix.at[mapping.map3Dto1D(x,y,yaw), mapping.map3Dto1D(pose[0],pose[1],pose[2])] = translation_matrix.at[mapping.map3Dto1D(x,y,yaw), mapping.map3Dto1D(pose[0],pose[1],pose[2])] +TRANSLATION_MATRIX_INITIAL_VALUE
translation_matrix.head()    

,0,1,2,3,4,5,6,7,8,9,...,3190,3191,3192,3193,3194,3195,3196,3197,3198,3199
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
#Create a dictionary for storing the transition states analysis
d ={}
#Our keys to the dictionary will look like (x_t, y_t, yaw_t, x_t+1, y_t+1, yaw_t+1)
#Go through all but last row since t+1 isnt defined there...
skipCount = 0
try:
    for index, row in df_discretized.iterrows():
        if(index == df_discretized.index[-1]):
            break
        if(df_discretized.at[index, 'ResetID'] != df_discretized.at[index + 1, 'ResetID']):
            skipCount += 1
            continue
        #Need to round here because Yaw data has floating point error
        key = (df_discretized.at[index, 'X'], df_discretized.at[index, 'Y'], round(df_discretized.at[index, 'Yaw'], 6),df_discretized.at[index+1, 'X'], df_discretized.at[index+1, 'Y'], round(df_discretized.at[index+1, 'Yaw'], 6) )

        if key in d:
            d[key] += 1
        else:
            d[key] = 1
except Exception as e:
    print e
    
print "[DEBUG] Skipped {} events".format(skipCount)
d

[DEBUG] Skipped 34 events


{(5.0, 8.0, 4.7124, 5.0, 9.0, 4.7124): 11,
 (5.0, 8.0, 2.3562, 5.0, 9.0, 2.3562): 14,
 (1.0, 9.0, 5.4978, 1.0, 9.0, 4.7124): 1,
 (12.0, 2.0, 2.3562, 12.0, 1.0, 2.3562): 7,
 (6.0, 13.0, 0.0, 7.0, 14.0, 0.0): 1,
 (3.0, 7.0, 0.0, 2.0, 6.0, 0.0): 2,
 (13.0, 3.0, 3.1416, 13.0, 2.0, 3.927): 1,
 (11.0, 13.0, 3.927, 11.0, 12.0, 3.927): 19,
 (9.0, 9.0, 0.7854, 8.0, 8.0, 0.7854): 1,
 (1.0, 5.0, 0.7854, 1.0, 5.0, 0.7854): 506,
 (7.0, 18.0, 3.1416, 6.0, 18.0, 3.1416): 14,
 (13.0, 18.0, 2.3562, 13.0, 18.0, 3.1416): 2,
 (13.0, 7.0, 0.0, 12.0, 6.0, 0.0): 1,
 (4.0, 16.0, 5.4978, 3.0, 17.0, 5.4978): 1,
 (8.0, 11.0, 3.1416, 8.0, 11.0, 3.1416): 169,
 (15.0, 18.0, 4.7124, 15.0, 18.0, 5.4978): 1,
 (14.0, 17.0, 2.3562, 14.0, 16.0, 2.3562): 15,
 (13.0, 13.0, 2.3562, 14.0, 12.0, 2.3562): 2,
 (11.0, 13.0, 3.1416, 11.0, 14.0, 3.1416): 11,
 (7.0, 13.0, 0.0, 8.0, 13.0, 0.0): 6,
 (8.0, 1.0, 3.1416, 8.0, 1.0, 3.1416): 413,
 (8.0, 18.0, 0.7854, 7.0, 18.0, 0.7854): 11,
 (16.0, 17.0, 3.1416, 17.0, 16.0, 3.1416): 2,
 (

In [16]:
mapping.map3Dto1D(0.0,0.0,4.0)

1600

In [17]:

#Fill in matrix with dictionary data


for key, value in d.iteritems():
    #we need to map yaw to an int state
    element_t = mapping.map3Dto1D(key[0], key[1], key[2])
    element_t_plus_1 = mapping.map3Dto1D(key[3], key[4], key[5])
    #Use the following to verify we the math above is fine
    if((mapping.checkValid1DMap(element_t)) & (mapping.checkValid1DMap(element_t_plus_1)) == 0 ):
        print "[ERROR] BAD MAPPING!"
    
    translation_matrix.at[element_t, element_t_plus_1] = value + translation_matrix.at[element_t, element_t_plus_1]
   # print("key = {}, elements = {}, {}".format(key, element_t, element_t_plus_1))


In [18]:
#Check sum of "events" per matrix

totalEvents = 0
for index,row in translation_matrix.iterrows():
    totalEvents += row.sum()
print("Total Events is {}".format(totalEvents))
print("Size of data point df is {}".format(df_discretized.size))

Total Events is 846055
Size of data point df is 5404630


In [19]:
#Divide the whole dataframe by number of data collections to get the probabilities.


magnitudeVector = pd.Series(0, index=range(n + 1))



for index, row in translation_matrix.iterrows():
    totalActionsInThisState = row.sum()
    magnitudeVector.iloc[index] = totalActionsInThisState
    if totalActionsInThisState == 0:
        continue
    translation_matrix.iloc[index] /= totalActionsInThisState

translation_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,3190,3191,3192,3193,3194,3195,3196,3197,3198,3199
0,0.125000,0.125000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.083333,0.083333,0.083333,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.083333,0.083333,0.083333,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.083333,0.083333,0.083333,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.083333,0.083333,0.083333,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
#validate the matrix (all rows == 1)
for index, row in translation_matrix.iterrows():
    if(abs(row.sum() - 1.0) > FLOAT_ERROR_TOLERANCE):
        print("[ERROR] Row probability not equal to one!")
        print(index)
        print(row.sum())


In [21]:
#Make matrix into CSV
translation_matrix.to_csv(data_prefix + 'translation_matrix_out.csv', encoding='utf-8', index=False)


In [22]:
#Make csv of the number of instances for each row
#magnitudeVector.to_csv(data_prefix + 'magnitude_vector_out.csv', encoding='utf-8', index=False)
